## Preprocessing

In [79]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Ignore tf warnings
import warnings
warnings.filterwarnings("ignore")

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Variables Analysis: 
Model Targets: IS_SUCCESSFUL

Model Features: APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, INCOME_AMT, ASK_AMT

In [80]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['NAME','EIN'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [81]:
# Drop Ask AMT outliers (over $1B)
application_df = application_df[application_df['ASK_AMT']<25000]

In [82]:
application_df['USE_CASE'].value_counts()

USE_CASE
Preservation     22026
ProductDev        4692
CommunityServ      344
Heathcare           17
Other                2
Name: count, dtype: int64

In [83]:
# Testing additional category drops
use_counts = application_df['USE_CASE'].value_counts()
uses_to_replace = use_counts[use_counts < 5000].index.tolist()

# Replace in dataframe
for x in uses_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(x,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

USE_CASE
Preservation    22026
Other            5055
Name: count, dtype: int64

In [84]:
application_df['ORGANIZATION'].value_counts()

ORGANIZATION
Trust           17605
Association      9239
Co-operative      208
Corporation        29
Name: count, dtype: int64

In [85]:
org_counts = application_df['ORGANIZATION'].value_counts()
orgs_to_replace = org_counts[org_counts < 10000].index.tolist()

# Replace in dataframe
for x in orgs_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(x,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

ORGANIZATION
Trust    17605
Other     9476
Name: count, dtype: int64

In [86]:
application_df['AFFILIATION'].value_counts()

AFFILIATION
CompanySponsored    13937
Independent         13099
Family/Parent          22
National               13
Regional                7
Other                   3
Name: count, dtype: int64

In [87]:
# use the variable name `application_types_to_replace`
affiliation_counts = application_df['AFFILIATION'].value_counts()
affiliation_to_replace = affiliation_counts[affiliation_counts < 10000].index.tolist()

# Replace in dataframe
for x in affiliation_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(x,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

AFFILIATION
CompanySponsored    13937
Independent         13099
Other                  45
Name: count, dtype: int64

In [88]:
# This ended up being irrelevant (actually reduced accuracy) but leaving in code to show it was attempted.
# # Optimization 1 - remove status=0 rows and then drop the status column
# application_df = application_df[application_df['STATUS']==1]
# application_df = application_df.drop(columns=['STATUS'])
# application_df.head()

In [89]:
# This ended up being irrelevant but leaving in code to show it was attempted.
# # Optimization idea - Bin the Ask Amounts to the same bins as the INCOME_AMT

# # Define bins and labels
# bins = [0, 9999, 24999, 99999, 499999, 5000000, 10000000, 50000000, float('inf')]
# labels = ['1-9999', '10000-24999', '25000-99999', '100000-499999', '1M-5M', '5M-10M', '10M-50M', '50M+']
# application_df['ASK_AMT_BINS'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels, right=False)
# application_df = application_df.drop(columns=['ASK_AMT'])
# application_df.head()


In [90]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            12
AFFILIATION                  3
CLASSIFICATION              66
USE_CASE                     2
ORGANIZATION                 2
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   1604
IS_SUCCESSFUL                2
dtype: int64

In [91]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts 

APPLICATION_TYPE
T3     20914
T4      1386
T19     1007
T6       922
T5       914
T8       680
T7       652
T10      508
T13       56
T9        21
T12       14
T2         7
Name: count, dtype: int64

In [92]:
# Optimization 3 - more binning 

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = app_counts[app_counts < 600].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       20914
T4        1386
T19       1007
T6         922
T5         914
T8         680
T7         652
Other      606
Name: count, dtype: int64

In [93]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

CLASSIFICATION
C1000    13082
C2000     4845
C1200     4129
C2100     1667
C3000     1585
         ...  
C1732        1
C1728        1
C4120        1
C1245        1
C2150        1
Name: count, Length: 66, dtype: int64

In [94]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = class_counts[class_counts < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    13082
C2000     4845
C1200     4129
Other     1773
C2100     1667
C3000     1585
Name: count, dtype: int64

In [95]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df).astype(int)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
5,1,5000,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [96]:
# Check for imbalanced outcomes to determine if stratify option is needed
application_df['IS_SUCCESSFUL'].value_counts()

IS_SUCCESSFUL
1    14314
0    12767
Name: count, dtype: int64

In [97]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [98]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [99]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 18
# hidden_nodes_layer3 = 50

nn = Sequential()
#nn = tf.keras.models.Sequential()

# First hidden layer using leaky relu
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features))
nn.add(LeakyReLU(alpha=0.01))  # Add LeakyReLU as a separate layer

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2))
nn.add(LeakyReLU(alpha=0.01))  # Add LeakyReLU as a separate layer

# Leaving some additional optimization attempts commented out here as they were either unnecessary or worse.
# Third hidden layer
#nn.add(Dense(units=hidden_nodes_layer3))
#nn.add(LeakyReLU(alpha=0.02))  # Add LeakyReLU as a separate layer

# First hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
#nn.add(Dense(units=1, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 24)                840       
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 24)                0         
                                                                 
 dense_10 (Dense)            (None, 18)                450       
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 18)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 19        
                                                                 
Total params: 1309 (5.11 KB)
Trainable params: 1309 (5.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [100]:
# Compile the model
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.compile(loss="mean_squared_error", optimizer="adamax", metrics=["accuracy"])

In [101]:
# Create a callback that saves the model's weights every five epochs.
checkpoint_path = "training/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq='epoch',
    period=5)


In [102]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20, callbacks=[cp_callback])

Epoch 1/20


635/635 [==============================] - 1s 725us/step - loss: 0.2059 - accuracy: 0.7055
Epoch 2/20
635/635 [==============================] - 0s 723us/step - loss: 0.1849 - accuracy: 0.7388
Epoch 3/20
635/635 [==============================] - 0s 717us/step - loss: 0.1823 - accuracy: 0.7412
Epoch 4/20
635/635 [==============================] - 0s 715us/step - loss: 0.1811 - accuracy: 0.7429
Epoch 5/20
619/635 [============================>.] - ETA: 0s - loss: 0.1807 - accuracy: 0.7420
Epoch 5: saving model to training\cp-0005.ckpt
635/635 [==============================] - 1s 788us/step - loss: 0.1803 - accuracy: 0.7427
Epoch 6/20
635/635 [==============================] - 0s 740us/step - loss: 0.1799 - accuracy: 0.7431
Epoch 7/20
635/635 [==============================] - 0s 723us/step - loss: 0.1794 - accuracy: 0.7444
Epoch 8/20
635/635 [==============================] - 0s 737us/step - loss: 0.1790 - accuracy: 0.7432
Epoch 9/20
635/635 [==============================] - 0s 720us/

In [103]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

212/212 - 0s - loss: 0.1751 - accuracy: 0.7525 - 188ms/epoch - 888us/step
Loss: 0.17509645223617554, Accuracy: 0.752473771572113


In [104]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')